In [ ]:
### Part 1: Import relevant libraries and conduct preliminary data exploration ###

In [3]:
# Install pandasql

pip install pandasql

  Using cached pandasql-0.7.3-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import libraries

import pandas as pd
from pandasql import sqldf

In [10]:
# Set dimension restrictions

pd.set_option('display.max_rows', 50)
pd.set_option('display.min_rows', 50)
pd.set_option('display.max_columns', 20)

In [6]:
# Read in first dataset

death_df = pd.read_csv('covid_deaths.csv')

In [7]:
# Read in second dataset

vacc_df = pd.read_csv('covid_vaccinations.csv')

/var/folders/80/0lqzpj_j1wb1msvsf1pxm37r0000gn/T/ipykernel_8992/2061577541.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  vacc_df = pd.read_csv('covid_vaccinations.csv')


In [11]:
# Explore the likelihood of someone in the United States dying if they contract Covid-19. Do this by creating a 
# column called "Death_Percentage" that is the quotient of total deaths divided by total cases.

q1 = """
    SELECT location, date, total_cases, total_deaths, (total_deaths/total_cases)*100 AS Death_Percentage
    FROM death_df
    WHERE location LIKE '%states%'
    AND location NOT LIKE '%virgin islands%'
    AND continent IS NOT null 
    ORDER BY 1, 2;
    """

print(sqldf(q1))

          location        date  total_cases  total_deaths  Death_Percentage
0    United States  2020-01-22          1.0           NaN               NaN
1    United States  2020-01-23          1.0           NaN               NaN
2    United States  2020-01-24          2.0           NaN               NaN
3    United States  2020-01-25          2.0           NaN               NaN
4    United States  2020-01-26          5.0           NaN               NaN
5    United States  2020-01-27          5.0           NaN               NaN
6    United States  2020-01-28          5.0           NaN               NaN
7    United States  2020-01-29          6.0           NaN               NaN
8    United States  2020-01-30          6.0           NaN               NaN
9    United States  2020-01-31          8.0           NaN               NaN
10   United States  2020-02-01          8.0           NaN               NaN
11   United States  2020-02-02          8.0           NaN               NaN
12   United 

In [12]:
# Determine the maximum, minimum, and average "Death_Percentage" in the United States

q2 = """
    SELECT MAX((total_deaths/total_cases)*100) AS Max_Death_Percentage, 
    MIN((total_deaths/total_cases)*100) AS Min_Death_Percentage,
    AVG((total_deaths/total_cases)*100) AS Avg_Death_Percentage
    FROM death_df
    WHERE location LIKE '%states%'
    AND location NOT LIKE '%virgin islands%'
    AND continent IS NOT null;
    """

print(sqldf(q2))

   Max_Death_Percentage  Min_Death_Percentage  Avg_Death_Percentage
0             10.909091              1.154383              2.310935


In [ ]:
# Analysis: In the United States from 1/22/20 to 7/7/22, a person's likelihood of dying after contracting Covid-19 
# peaked at about 11% and hit its lowest point at about 1.15%. The peak occurred in March 2020, while the 
# trough occured in July 2022. 

# (Note: It's important to point out that the percentage associated with the peak may be inaccurate as there was 
# limited data on infection rates and death rates at the time, as evidenced by the relatively small number of 
# recorded cases and deaths in March 2020.)

# The average likelihood of death for someone who became infected with Covid-19 between 1/22/20 and 7/7/22
# was approximately 2.3%.

In [13]:
# Explore what percentage of the U.S. population was infected with Covid-19 between 1/22/20 and 7/7/22. Do this by 
# creating a column called "Total_Pop_Infection_Rate" that is the quotient of total cases divided by the population.

q3 = """
    Select location, date, population, total_cases, CAST((total_cases/population)*100 AS VARCHAR(40)) AS Percent_Population_Infected
    FROM death_df
    WHERE location LIKE '%states%'
    AND location NOT LIKE '%virgin islands%'
    AND continent IS NOT null 
    ORDER BY 1, 2;
    """

print(sqldf(q3))

          location        date   population  total_cases  \
0    United States  2020-01-22  332915074.0          1.0   
1    United States  2020-01-23  332915074.0          1.0   
2    United States  2020-01-24  332915074.0          2.0   
3    United States  2020-01-25  332915074.0          2.0   
4    United States  2020-01-26  332915074.0          5.0   
5    United States  2020-01-27  332915074.0          5.0   
6    United States  2020-01-28  332915074.0          5.0   
7    United States  2020-01-29  332915074.0          6.0   
8    United States  2020-01-30  332915074.0          6.0   
9    United States  2020-01-31  332915074.0          8.0   
10   United States  2020-02-01  332915074.0          8.0   
11   United States  2020-02-02  332915074.0          8.0   
12   United States  2020-02-03  332915074.0         11.0   
13   United States  2020-02-04  332915074.0         11.0   
14   United States  2020-02-05  332915074.0         11.0   
15   United States  2020-02-06  33291507

In [14]:
# Determine the maximum and average infection rates in the United States during the pandemic between 1/22/20 and 7/7/22.

q4 = """
    Select MAX((total_cases/population)*100) AS Max_Infection_Rate, 
    AVG((total_cases/population)*100) AS Avg_Infection_Rate
    FROM death_df
    WHERE location LIKE '%states%'
    AND location NOT LIKE '%virgin islands%'
    AND continent IS NOT null;
    """

print(sqldf(q4))

   Max_Infection_Rate  Avg_Infection_Rate
0           26.547788             9.99088


In [ ]:
# Analysis: The average infection rate in the United States from 1/22/20 to 7/7/22 is about 10%, while the maximum 
# infection rate is about 26.6%. 

# The output for query 3 (q3) shows that the infection rate has increased as time has progressed, with the maximum 
# infection rate from 1/22/20 to 7/7/22 occuring in July 2022. 

In [15]:
# Order countries according to infection rates as a proportion of their populations, with countries exhibiting
# the highest infection rates as a proportion of their populations appearing at the top of the query. 

q5 = """
    SELECT location, population, MAX(total_cases) AS Highest_Infection_Count,  MAX((total_cases/population))*100 AS Percent_Population_Infected
    FROM death_df
    GROUP BY location, population
    ORDER BY Percent_Population_Infected DESC;
    """

print(sqldf(q5))

                         location    population  Highest_Infection_Count  \
0                  Faeroe Islands  4.905300e+04                  34658.0   
1                       Gibraltar  3.369100e+04                  19796.0   
2                         Andorra  7.735400e+04                  44671.0   
3                          Cyprus  8.960050e+05                 515596.0   
4                         Denmark  5.813302e+06                3185838.0   
5                      San Marino  3.401000e+04                  18361.0   
6                         Iceland  3.687920e+05                 196527.0   
7                Falkland Islands  3.528000e+03                   1831.0   
8                        Portugal  1.016792e+07                5234600.0   
9                        Slovenia  2.078723e+06                1046425.0   
10                        Austria  9.043072e+06                4529939.0   
11      Saint Pierre and Miquelon  5.771000e+03                   2779.0   
12          

In [16]:
# Focus on infection rates as a proportion of population on the Faroe Islands (the country with the highest infection
# count, according to query 5)

q6 = """
    Select location, date, population, total_cases, CAST((total_cases/population)*100 AS VARCHAR(40)) AS Percent_Population_Infected
    FROM death_df
    WHERE location LIKE '%Faeroe Islands%'
    AND continent IS NOT null 
    ORDER BY 1, 2;
    """

print(sqldf(q6))

           location        date  population  total_cases  \
0    Faeroe Islands  2020-03-04     49053.0          1.0   
1    Faeroe Islands  2020-03-05     49053.0          1.0   
2    Faeroe Islands  2020-03-06     49053.0          1.0   
3    Faeroe Islands  2020-03-07     49053.0          1.0   
4    Faeroe Islands  2020-03-08     49053.0          2.0   
5    Faeroe Islands  2020-03-09     49053.0          2.0   
6    Faeroe Islands  2020-03-10     49053.0          2.0   
7    Faeroe Islands  2020-03-11     49053.0          2.0   
8    Faeroe Islands  2020-03-12     49053.0          2.0   
9    Faeroe Islands  2020-03-13     49053.0          3.0   
10   Faeroe Islands  2020-03-14     49053.0          9.0   
11   Faeroe Islands  2020-03-15     49053.0         11.0   
12   Faeroe Islands  2020-03-16     49053.0         18.0   
13   Faeroe Islands  2020-03-17     49053.0         47.0   
14   Faeroe Islands  2020-03-18     49053.0         58.0   
15   Faeroe Islands  2020-03-19     4905

In [ ]:
# Analysis: The Faroe Islands' maximum infection rate is about 70.65%. The country's infection rate increased 
# fairly consistently from 1/22/20 to 7/7/22. According to the dataset, the Faroe Islands' infection rate plateaued 
# at 70.65% (i.e., the maximum infection rate), indicating that new data regarding total cases probably hadn't  
# been published recently at the time of this analysis. 

In [17]:
# Explore which countries experienced the most Covid-related deaths

q7 = """
    SELECT location, MAX(cast(Total_deaths as int)) as Total_Death_Count
    FROM death_df
    WHERE continent IS NOT null
    GROUP BY location
    ORDER BY Total_Death_Count DESC;
    """

print(sqldf(q7))

                         location  Total_Death_Count
0                   United States          1020262.0
1                          Brazil           673073.0
2                           India           525343.0
3                          Russia           373732.0
4                          Mexico           325976.0
5                            Peru           213623.0
6                  United Kingdom           180898.0
7                           Italy           168864.0
8                       Indonesia           156776.0
9                          France           150012.0
10                        Germany           141758.0
11                           Iran           141420.0
12                       Colombia           140202.0
13                      Argentina           129109.0
14                         Poland           116449.0
15                        Ukraine           112459.0
16                          Spain           108259.0
17                   South Africa           10

In [19]:
# Explore which countries had the highest death rate as a proportion of their population

q8 = """
    SELECT location, population, MAX(total_deaths) AS Total_Death_Count,  MAX((total_deaths/population))*100 AS Percent_Population_Deaths
    FROM death_df
    GROUP BY location, population
    ORDER BY Percent_Population_Deaths DESC;
    """

print(sqldf(q8))

                         location    population  Total_Death_Count  \
0                            Peru  3.335942e+07           213623.0   
1                        Bulgaria  6.896655e+06            37264.0   
2          Bosnia and Herzegovina  3.263459e+06            15809.0   
3                         Hungary  9.634162e+06            46661.0   
4                 North Macedonia  2.082661e+06             9329.0   
5                      Montenegro  6.280510e+05             2729.0   
6                         Georgia  3.979773e+06            16844.0   
7                         Croatia  4.081657e+06            16096.0   
8                         Czechia  1.072455e+07            40330.0   
9                        Slovakia  5.449270e+06            20154.0   
10                        Romania  1.912777e+07            65755.0   
11                      Lithuania  2.689862e+06             9178.0   
12                     San Marino  3.401000e+04              115.0   
13                  

In [ ]:
# Analysis: The country with the most Covid-related deaths from 1/22/20 to 7/7/22 was the United States, with 1,020,262 
# deaths. Thus, according to this dataset, as of 7/7/20, about 0.31% of the United States' population had died as 
# a result of Covid-19. 

# The country with the highest death rate as a proportion of its population is Peru, with a death per population
# rate of about 0.64%.

In [20]:
# Determine the total number of Covid-related deaths worldwide and the percentage of the world's population
# that has died because of Covid

q9 = """
    SELECT SUM(new_cases) AS Total_Cases, SUM(cast(new_deaths AS int)) AS Total_Deaths, SUM(cast(new_deaths AS int))/SUM(New_Cases)*100 AS Death_Percentage
    FROM death_df
    WHERE continent IS NOT null
    ORDER BY 1, 2;
    """

print(sqldf(q9))

   Total_Cases  Total_Deaths  Death_Percentage
0  552401077.0       6305479          1.141468


In [ ]:
# Analysis: Between 1/22/20 and 7/7/22, there have been 6,305,479 Covid-related deaths, and about 1.14% of the 
# world's population has died as a result of Covid-19. 

In [21]:
# Join the death_df and vacc_df datasets to determine what percentage of the U.S. population received at least 
# one Covid vaccination between 1/22/20 and 7/7/22.

q10 = """
SELECT dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations,
SUM(CAST(vac.new_vaccinations AS int)) OVER (PARTITION BY dea.location ORDER BY dea.location, dea.date) AS Rolling_People_Vaccinated
FROM death_df dea
JOIN vacc_df vac
	ON dea.location = vac.location
	AND dea.date = vac.date
WHERE dea.location LIKE '%states%'
AND dea.location NOT LIKE '%virgin islands%'
AND dea.continent IS NOT null
ORDER BY 2,3;
"""

print(sqldf(q10))

         continent       location        date   population  new_vaccinations  \
0    North America  United States  2020-01-22  332915074.0               NaN   
1    North America  United States  2020-01-23  332915074.0               NaN   
2    North America  United States  2020-01-24  332915074.0               NaN   
3    North America  United States  2020-01-25  332915074.0               NaN   
4    North America  United States  2020-01-26  332915074.0               NaN   
5    North America  United States  2020-01-27  332915074.0               NaN   
6    North America  United States  2020-01-28  332915074.0               NaN   
7    North America  United States  2020-01-29  332915074.0               NaN   
8    North America  United States  2020-01-30  332915074.0               NaN   
9    North America  United States  2020-01-31  332915074.0               NaN   
10   North America  United States  2020-02-01  332915074.0               NaN   
11   North America  United States  2020-

In [22]:
 # Join the death_df and vacc_df datasets to determine what percentage of the U.S. population became fully vaccinated 
# between 1/22/20 and 7/7/22.

q11 = """
SELECT MAX(vac.people_fully_vaccinated/dea.population)
FROM death_df dea
JOIN vacc_df vac
	ON dea.location = vac.location
	AND dea.date = vac.date
WHERE dea.location LIKE '%states%'
AND dea.location NOT LIKE '%virgin islands%'
AND dea.continent IS NOT null;
"""

print(sqldf(q11))

   MAX(vac.people_fully_vaccinated/dea.population)
0                                         0.668205


In [ ]:
# Analysis: As of 7/7/22, about 67% of U.S. population was fully vaccinated against Covid-19. 